In [1]:
code_search_path=["/home/byzerllm/softwares/byzer-retrieval-lib/"]
env_vars = {"JAVA_HOME": "/home/byzerllm/softwares/jdk-21",
            "PATH":"/home/byzerllm/softwares/jdk-21/bin:/home/byzerllm/.rvm/gems/ruby-3.2.2/bin:/home/byzerllm/.rvm/gems/ruby-3.2.2@global/bin:/home/byzerllm/.rvm/rubies/ruby-3.2.2/bin:/home/byzerllm/.rbenv/shims:/home/byzerllm/.rbenv/bin:/home/byzerllm/softwares/byzer-lang-all-in-one-linux-amd64-3.3.0-2.3.7/jdk8/bin:/usr/local/cuda/bin:/usr/local/cuda/bin:/home/byzerllm/.rbenv/shims:/home/byzerllm/.rbenv/bin:/home/byzerllm/miniconda3/envs/byzerllm-dev/bin:/home/byzerllm/miniconda3/condabin:/home/byzerllm/.local/bin:/home/byzerllm/bin:/usr/local/cuda/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/byzerllm/.rvm/bin:/home/byzerllm/.rvm/bin"}

import os
os.environ["RAY_DEDUP_LOGS"] = "0" 

from byzerllm.utils.client import Templates

import ray
from byzerllm.utils.retrieval import ByzerRetrieval
from byzerllm.utils.client import ByzerLLM,LLMRequest,LLMResponse,LLMHistoryItem,InferBackend
from byzerllm.records import SearchQuery

ray.init(address="auto",namespace="default",ignore_reinit_error=True,
                 job_config=ray.job_config.JobConfig(code_search_path=code_search_path,
                                                      runtime_env={"env_vars": env_vars})
                 )   

# ray.init(address="auto",namespace="default")   

retrieval = ByzerRetrieval()
retrieval.launch_gateway()

llm = ByzerLLM()
chat_model_name = "chat"

# llm.undeploy(chat_model_name)
# llm.undeploy("emb")


# model_type = "deepseek"
# model_type = "openbuddy"
# model_type = "Qwen"
model_type = "zephyr"

if model_type == "deepseek":
    model_location="/home/byzerllm/models/deepseek2-llm-67b-chat"
    max_model_len = 4096
    gpus_per_worker = 8

if model_type == "llama2":
    model_location="/home/byzerllm/models/openbuddy-llama2-70b-v10.1-bf16"
    max_model_len = 4000
    gpus_per_worker = 8

if model_type == "Qwen":
    llm.setup_template("chat",Templates.qwen())
    model_location="/home/byzerllm/models/Qwen-72B-Chat"
    max_model_len = 24000
    gpus_per_worker = 8

if model_type == "zephyr": 
    model_location="/home/byzerllm/models/openbuddy-zephyr-7b-v14.1"   
    max_model_len = 24000
    gpus_per_worker = 4

llm.setup_max_model_length("chat",max_model_len)

if not llm.is_model_exist("chat"):
    llm.setup_gpus_per_worker(gpus_per_worker).setup_num_workers(1).setup_infer_backend(InferBackend.VLLM)
    llm.deploy(
        model_path=model_location,
        pretrained_model_type="custom/auto",
        udf_name=chat_model_name,
        infer_params={"backend.max_num_batched_tokens":24000,
                      "backend.max_model_len":max_model_len}
    )


if not llm.is_model_exist("emb"):
    llm.setup_gpus_per_worker(0.4).setup_num_workers(2).setup_infer_backend(InferBackend.Transformers)
    llm.deploy(
        model_path="/home/byzerllm/models/bge-large-zh",
        pretrained_model_type="custom/bge",
        udf_name="emb",
        infer_params={}
    )    

llm.setup_default_model_name(chat_model_name) 
llm.setup_default_emb_model_name("emb") 

# ray.kill(ray.get_actor("data_analysis"))
# 如果存储集群不存在，启动一个存储集群
if not retrieval.is_cluster_exists("data_analysis"):
    builder = retrieval.cluster_builder()
    builder.set_name("data_analysis").set_location("/tmp/data_analysis").set_num_nodes(2).set_node_cpu(1).set_node_memory("3g")
    builder.set_java_home(env_vars["JAVA_HOME"]).set_path(env_vars["PATH"]).set_enable_zgc()
    builder.start_cluster()    

def show_code(lang,code_string):
    from IPython.display import display, Markdown    
    display(Markdown("```{}\n{}\n```".format(lang,code_string)))


def show_text(msg):
    from IPython.display import display, Markdown
    display(Markdown("```{}\n{}\n```".format("text",msg))) 

def show_image(content):
    from IPython.display import display, Image
    import base64             
    img = Image(base64.b64decode(content))
    display(img)    
    

2023-12-14 21:15:57,069	INFO worker.py:1489 -- Connecting to existing Ray cluster at address: 192.168.1.248:6379...
2023-12-14 21:15:57,081	INFO worker.py:1664 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 


Failed to look up actor with name 'emb'. This could because 1. You are trying to look up a named actor you didn't create. 2. The named actor died. 3. You did not use a namespace matching the namespace of the actor.
(UDFWorker pid=534687, ip=192.168.1.243) MODEL[emb] Init Model,It may take a while.
(UDFWorker pid=534687, ip=192.168.1.243) MODEL[emb]  Local mode: Load model from local path (/home/byzerllm/models/bge-large-zh), consume the model server to prevent socket server leak.
(UDFWorker pid=534688, ip=192.168.1.243) MODEL[emb] Init Model,It may take a while.
(UDFWorker pid=534688, ip=192.168.1.243) MODEL[emb]  Local mode: Load model from local path (/home/byzerllm/models/bge-large-zh), consume the model server to prevent socket server leak.
(UDFWorker pid=534688, ip=192.168.1.243) MODEL[emb] Successful to init model, time taken:3.4095799922943115s


(UDFWorker pid=534687, ip=192.168.1.243) MODEL[emb] Successful to init model, time taken:3.446164608001709s


In [2]:
llm.emb("emb",LLMRequest(instruction="我试试看能不能把这句话转换成向量"))

[LLMResponse(output=[0.003487164154648781, -0.009687656536698341, -0.026311276480555534, -0.004805233795195818, 0.03333209827542305, 0.014389371499419212, 0.017725428566336632, -0.008188240230083466, 0.008683566004037857, 0.013795615173876286, -0.004662488587200642, -0.01721588335931301, 0.007674216292798519, -0.036708295345306396, -0.010143325664103031, 0.03057188168168068, 0.004054826684296131, 0.007601254619657993, 0.021168725565075874, -0.043669793754816055, -0.004535554442554712, -0.034899331629276276, -0.02656671404838562, -0.007852491922676563, -0.011296529322862625, 0.0011517584789544344, 0.003761491971090436, 0.036769669502973557, 0.005315430928021669, -0.0304831825196743, -0.03222383186221123, 0.03505197912454605, 0.016987627372145653, -0.0006790258339606225, 0.009743181988596916, 0.022959737107157707, -0.031860850751399994, 0.007448054384440184, -0.02996632643043995, -0.0014919526875019073, -0.013263804838061333, -0.007554023526608944, -0.01376862172037363, -0.03440831601619

In [3]:
from byzerllm.apps.agent import Agents

from byzerllm.apps.agent.user_proxy_agent import UserProxyAgent
from byzerllm.apps.agent.extensions.retrieval_agent import RetrievalAgent


retrieval_agent = Agents.create_local_agent(RetrievalAgent,"retrieval_agent",llm,retrieval,
                                   max_consecutive_auto_reply=1000,
                                   code_agent = None
                                   )

user = Agents.create_local_agent(UserProxyAgent,"user",llm,retrieval,
                                human_input_mode="NEVER",
                                max_consecutive_auto_reply=0)



In [5]:
user.initiate_chat(
retrieval_agent,
message={
    "content":"介绍下 Gluten",
    "metadata":{
        "file_path":"/home/byzerllm/projects/jupyter-workspace/aritcle.txt"
    }
},)


user (to retrieval_agent):

介绍下 Gluten

--------------------------------------------------------------------------------
retrieval_agent (to user):

Gluten是Apache Spark的一个本地化计算引擎，它结合了Spark的可扩展性框架、优化器、容错能力和Native执行库的高性能，以提高数据处理速度并降低计算成本。它可以与Velox或Clickhouse集成。BIGO大数据团队是参与该项目的研发团队之一，并已在生产环境中使用Gluten替代Spark的ETL工作负载，实现了成本节省。

--------------------------------------------------------------------------------
